In [1]:
# multivariate multi-step encoder-decoder lstm example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM, Bidirectional
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import EarlyStopping
import tensorflow_hub as hub
import numpy as np
import tensorflow_text
from sklearn.neighbors import NearestCentroid
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import pandas as pd
import itertools
from random import sample 
from keras.models import load_model
from numpy import array
import numpy as np
from gensim.models import Word2Vec, tfidfmodel


Using TensorFlow backend.
/home/omno/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/home/omno/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#dep

In [8]:

n_steps_in, n_steps_out = 4, 1
# covert into input/output
X = np.load('data/X.data.npy')
y = np.load('data/y.data.npy')
print(X.shape)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]


(135597, 4, 20)


In [13]:

# define model
model = Sequential()
model.add(Bidirectional(LSTM(512, activation='relu', input_shape=(n_steps_in, n_features), dropout=0.4, recurrent_dropout=0.2)))
model.add(RepeatVector(n_steps_out))
model.add(Bidirectional(LSTM(512, activation='relu', return_sequences=True, dropout=0.4, recurrent_dropout=0.2)))
model.add(TimeDistributed(Dense(n_features)))

model.compile(optimizer='adam', loss='mse')
# fit model
es = EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=5, restore_best_weights=True)
model.fit(X, y, batch_size=32, epochs=100, verbose=1, validation_split=0.33, callbacks=[es])
# demonstrate prediction

model.save('data/model.h5')

Train on 90849 samples, validate on 44748 samples
Epoch 1/100
90849/90849 [==============================] - 330s 4ms/step - loss: 0.1207 - val_loss: 0.1105
Epoch 2/100
90849/90849 [==============================] - 332s 4ms/step - loss: 0.1158 - val_loss: 0.1081
Epoch 3/100
90849/90849 [==============================] - 292s 3ms/step - loss: 0.1145 - val_loss: 0.1082
Epoch 4/100
90849/90849 [==============================] - 270s 3ms/step - loss: 0.1137 - val_loss: 0.1068
Epoch 5/100
90849/90849 [==============================] - 268s 3ms/step - loss: 0.1130 - val_loss: 0.1063
Epoch 6/100
90849/90849 [==============================] - 268s 3ms/step - loss: 0.1126 - val_loss: 0.1059
Epoch 7/100
90849/90849 [==============================] - 266s 3ms/step - loss: 0.1122 - val_loss: 0.1055
Epoch 8/100
90849/90849 [==============================] - 268s 3ms/step - loss: 0.1118 - val_loss: 0.1052
Epoch 9/100
90849/90849 [==============================] - 268s 3ms/step - loss: 0.1116 - val_

In [2]:
w2v = Word2Vec.load("data/word2vec.model")
recom_model= load_model('data/model.h5')



In [3]:
X = np.load('data/X.data.npy')
y = np.load('data/y.data.npy')

X_org= np.load('data/X_org.data.npy')
Y_org= np.load('data/y_org.data.npy')

In [4]:
def decode_output(model,item, top_n=-1):
    result=model.wv.most_similar(item)
    result= [i[0] for i in result][0:top_n]
    return result
    
def inference(seq, e_model):
    return [decode_output(e_model, seq[0])]#,decode_output(e_model,seq[1])] 

In [6]:
correct=0
freq=0

#Model Inference
Y_pred= recom_model.predict(X, verbose=1)

for i in range(len(Y_pred)):
    #predicted embedings top_n match
    results=inference(Y_pred[i],w2v)
    if Y_org[i][0] in results[0]:
        correct+=1
    freq+=1

print("Accuracy:",(correct/freq)*100)

135597/135597 [==============================] - 53s 390us/step
Accuracy: 23.475445621953288
